In [1]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd
from biomarker.data_collection import *
import numpy as np

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
NUM_TEST = 15
SEED = 1

In [3]:
np.random.seed(SEED)

In [4]:
excel = parse_master_file()

In [5]:
L = get_filename_list(excel['Associated data'])

In [6]:
x1 = create_x1_matrix(L)

In [7]:
x1.shape

(191, 250)

In [8]:
x4 = create_x4_matrix(L)

In [9]:
x4.shape

(191, 150)

In [10]:
x5 = create_x5_matrix(L)

In [11]:
x5.shape

(191, 200)

In [12]:
x6 = create_x6_matrix(L)

In [13]:
x6.shape

(191, 300)

In [14]:
x7 = create_x7_matrix(L)

In [15]:
x7.shape

(191, 150)

In [16]:
y = excel['Output: logK'].values
x10_x17 = excel.iloc[:, 3:-2]

In [17]:
master = prepare_master(x10_x17)

(191, 8)


In [18]:
test_idxs = np.random.randint(0,len(y),NUM_TEST)
train_idxs = np.ones(y.shape,dtype=bool)
train_idxs[test_idxs] = False
y_train = y[train_idxs]
y_test = y[test_idxs]

In [19]:
x1_approx_train = linear_regression_approx(x1[train_idxs], y_train)
x4_approx_train = linear_regression_approx(x4[train_idxs], y_train)
x5_approx_train = linear_regression_approx(x5[train_idxs], y_train)
x6_approx_train = linear_regression_approx(x6[train_idxs], y_train)
x7_approx_train = linear_regression_approx(x7[train_idxs], y_train)

/Users/tgebhart/.virtualenvs/biomarker_2/lib/python2.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [20]:
master_train = master[train_idxs]
master_test = master[test_idxs]

In [21]:
regr = linear_model.LinearRegression()
all_xs_train = np.column_stack((x1_approx_train, x4_approx_train, x5_approx_train, x6_approx_train, x7_approx_train, master_train))
regr.fit(all_xs_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [22]:
x1_approx_test = linear_regression_approx(x1[test_idxs], y_test)
x4_approx_test = linear_regression_approx(x4[test_idxs], y_test)
x5_approx_test = linear_regression_approx(x5[test_idxs], y_test)
x6_approx_test = linear_regression_approx(x6[test_idxs], y_test)
x7_approx_test = linear_regression_approx(x7[test_idxs], y_test)

In [23]:
all_xs_test = np.column_stack((x1_approx_test, x4_approx_test, x5_approx_test, x6_approx_test, x7_approx_test, master_test))

In [24]:
predictions = regr.predict(all_xs_test)

In [25]:
predictions - y_test

array([0.0021388453895383464, -0.06469526105577428, 0.10805559343641313,
       0.25020036882703867, -0.02185163312608651, 0.020208913748913204,
       -0.06155378156358671, -0.052487619454210765, -0.08566633039171201,
       -0.018119943672961547, 0.21760027117078806, -0.031091135079211796,
       0.21949236101453806, -0.046351144844837044, 0.28005418962781947],
      dtype=object)

In [26]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, predictions))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, predictions))

# # Plot outputs
# plt.scatter(diabetes_X_test, y_test,  color='black')
# plt.plot(diabetes_X_test, predictions, color='blue', linewidth=3)

# plt.xticks(())
# plt.yticks(())

# plt.show()

('Coefficients: \n', array([-1.69270883e+00,  2.54537047e+10, -2.54521638e+10, -6.11526588e-01,
       -1.54093337e+06, -1.34199654e-01,  1.46679158e-01,  3.81601973e-02,
       -1.37723315e-01, -7.27839779e-03, -3.94068826e-03,  5.61336771e-02,
       -4.37276352e-01, -2.61084688e-01, -5.55421303e-03,  6.61076477e-04,
        7.33586176e+05,  7.33586400e+05,  7.33586211e+05,  7.33586212e+05,
        2.19729426e-01, -7.33586218e+05, -7.33586142e+05, -7.33586088e+05,
       -7.33586068e+05, -7.33586293e+05, -7.33586068e+05]))
Mean squared error: 0.02
Variance score: 1.00
